# 1. Cleaning & EDA

In [1]:
import os
import re
import pandas as pd
import numpy as np
import warnings

In [2]:
#pull all files from folder and make single dataframe
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

folder = os.listdir(path='./twitter_data/')
df = pd.DataFrame()

for filename in folder:
    if 'twitter_user_' in str(filename):
        df = df.append(pd.read_csv(f'./twitter_data/{filename}', index_col=0))
        warnings.simplefilter("ignore")


/var/folders/vs/w7rl81xx4v3dmfdm5sty1pl40000gn/T/ipykernel_27532/2697461994.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(f'./twitter_data/{filename}', index_col=0))


In [3]:
df.reset_index(inplace=True, drop=True)

In [4]:
df.shape

(4023, 2)

In [5]:
df['Tweet'].shape[0]

4023

In [6]:
#create new column
df['new_tweet'] = df['Tweet']

In [7]:
#remove all usernames
# https://stackoverflow.com/questions/50830214/remove-usernames-from-twitter-data-using-python
no_usernames = []

for ind in range(0,df['Tweet'].shape[0]):
    no_usernames.append(re.sub('@[\w]+','', df['Tweet'][ind]))



In [8]:
df['new_tweet'] = no_usernames

In [9]:
df[['new_tweet']].head(2)

,new_tweet
0,"RT : First and foremost, dropped this gem - '..."
1,RT : Finally got around to listening to this i...


In [10]:
# remove all URLs
#https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python

no_urls = []

for ind in range(0, df['new_tweet'].shape[0]):
    no_urls.append(re.sub(r'http\S+', '', df['new_tweet'][ind]))

df['no_urls'] = no_urls

In [22]:
#verify that URLs were removed
df[['no_urls','new_tweet']].tail(6)

,no_urls,new_tweet
4017,First day of December in Miami isn’t too bad ;),First day of December in Miami isn’t too bad ;...
4018,Great dinner in Miami tonight! and an...,Great dinner in Miami tonight! and an...
4019,RT : I asked to let me on to his podcast beca...,RT : I asked to let me on to his podcast beca...
4020,check your email!,check your email!
4021,,https://t.co/Uif4dzkjXR
4022,,


In [13]:
# save whether a tweet is a retweet or not
df['retweet'] = np.where(df.no_urls.str.contains("RT :"), 1, 0)

In [18]:
df[['no_urls','retweet']].head()

,no_urls,retweet
0,"RT : First and foremost, dropped this gem - '...",1
1,RT : Finally got around to listening to this i...,1
2,RT : After thousands of private requests we ar...,1
3,RT : 1/ NEW\n\nLux Q1 2022 Letter to LPs \n\n“...,1
4,Not bad! The inverse\n\nEDGES give WEDGES\n\ni...,0


In [16]:
#save anonymized tweets with no URLs
#df[['no_urls','retweet']].to_csv('./data/anon_tweets.csv', index=0)

In [17]:
df['retweet'].value_counts(normalize=True)

0    0.742232
1    0.257768
Name: retweet, dtype: float64

The sample of ~4,000 tweets is split about 75/25 into original tweets and retweets. 